In [1]:
#importing data
claims_df = spark.read.format("json").load("s3://lhn-capstone/claims.json",header = "true")

VBox()

Starting Spark application


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
lhn_claims = claims_df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
lhn_claims = lhn_claims.filter(lhn_claims.Claim_Processed_On!='null')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:

from pyspark.sql.functions import to_timestamp
lhn_claims = lhn_claims.withColumn("Claim_Processed_On",to_timestamp("Claim_Processed_On", 'dd-MM-yyyy HH:mm'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:

from pyspark.sql.functions import to_timestamp,year
lhn_claims = lhn_claims.withColumn('Year',year(to_timestamp('Claim_Processed_On', 'dd-MM-yyyy HH:mm')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:

from pyspark.sql.functions import to_timestamp,date_format
from pyspark.sql.functions import col


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
lhn_claims2 = lhn_claims.withColumn("Claim_Processed_On",to_timestamp(col("Claim_Processed_On"))).withColumn("Month", date_format(col("Claim_Processed_On"), "M"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:

from pyspark.sql.functions import to_timestamp


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
lhn_claims2 = lhn_claims2.withColumn("Claim_Logged_On",to_timestamp("Claim_Logged_On", 'dd-MM-yyyy HH:mm'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:

lhn_claims3 = lhn_claims2.withColumn("vehicle",lhn_claims2["vehicle"].cast('double'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
lhn_claims3.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------------------+-------------------+--------------+---------------------+---------------+---------------+-------------+----------------+--------------------+-----------------+--------------+--------------------+-------+---------------------------+-----------------------+--------+-------+---------+----+-----+
| ClaimID|    Claim_Logged_On| Claim_Processed_On|Claim_Rejected|authorities_contacted|bodily_injuries| collision_type|incident_city|   incident_date|   incident_location|incident_severity|incident_state|       incident_type| injury|number_of_vehicles_involved|police_report_available|property|vehicle|witnesses|Year|Month|
+--------+-------------------+-------------------+--------------+---------------------+---------------+---------------+-------------+----------------+--------------------+-----------------+--------------+--------------------+-------+---------------------------+-----------------------+--------+-------+---------+----+-----+
| 7301700|2018-01-24 23:16:0

In [12]:

lhn_claims4 = lhn_claims3.withColumn("Total Claim Amount",lhn_claims3.property+lhn_claims3.injury+lhn_claims3.vehicle)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:

from pyspark.sql.functions import *
lhn_claims5 = lhn_claims4.withColumn("Processing_time",datediff(col("Claim_Processed_On"),col("Claim_Logged_On")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:

from pyspark.sql import functions as F
claims_metric = lhn_claims5.groupBy('Year','Month').agg(F.sum('Total Claim Amount').alias('Total_Claims_Amount'), F.mean('Processing_time').alias('Average_Processing_time')).sort('Year','Month')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
#exporting metric to s3 bucket
claims_metric.write.parquet("s3://lhn-capstone/ClaimsMetricpyspark.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…